In [ ]:
!pip install ipython-autotime
%load_ext autotime

from google.colab import drive
import os
from datetime import datetime

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/DL and CV')

Mounted at /content/drive
time: 23 s (started: 2022-05-07 00:21:32 +00:00)


In [ ]:
#-- Check the running GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-54285c3c-3c93-7d77-1a5f-297613c4f30d)
time: 117 ms (started: 2022-05-07 00:21:55 +00:00)


In [ ]:
#-- Running Tensorboard within google colab noebook
%load_ext tensorboard
%tensorboard --logdir="./models/MNIST/runs" #-- Read logs from MNIST directory
# %tensorboard --logdir="./models/CIFAR/runs" #-- Read logs from CIFAR directory

# Imports

In [ ]:
import time
import random
from tqdm.notebook import tqdm_notebook
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from torchvision.models import vgg19_batch_norm
from torch.utils.tensorboard import SummaryWriter

time: 3.83 s (started: 2022-05-07 00:22:00 +00:00)


In [ ]:
def seed():
  """ This function is used when running any cell to make sure all the seed are the same"""
  rand_seed = 0
  random.seed(rand_seed)
  os.environ['PYTHONHASHSEED'] = str(rand_seed)
  np.random.seed(rand_seed)
  torch.manual_seed(rand_seed)
  torch.cuda.manual_seed(rand_seed)
  torch.cuda.manual_seed_all(rand_seed) # if you are using multi-GPU.
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

time: 3.68 ms (started: 2022-05-07 00:22:04 +00:00)


# Train & Validate

In [ ]:
seed()
def model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters):
  #-- Build a lamba function to get the current time stamp when it is called
  ts = lambda x: x.now().strftime('%d_%m_%Y__%H:%M:%S')  
  #-- Change the VGG output layer to 10 to match the number of MNIST and CIFAR classes
  model.classifier[6] = nn.Linear(4096, 10)  
  #-- Pass model to to cuda
  model.to(device)  
  # Add VGG model graph to tensorboard
  # writer.add_graph(model, images)  
  loss_criterion = nn.CrossEntropyLoss()  

  print('###################### Training {} {} model for {} epochs ######################'.format(model_name, parameters, epochs))
  #-- Set the lowest validation loss to positive infinity 
  best_val_loss = np.inf
  #-- loop over the dataset for a number of epochs 
  for epoch in range(epochs):
      ###################
      # train the model #
      ###################
      #-- Initiate train loss, correct training , epoch loss, epoch accuracy, and epoch_accuracy
      train_loss = 0.0
      train_correct = 0
      total = 0 ####################### CHANGE names
      epoch_loss = 0.0
      epoch_acc = 0.0
      # start_time = time.time()
      model.train()
      for i, (images, labels) in tqdm_notebook(enumerate(train_loader)): 
          #-- Pass images and labels to cuda
          images, labels = images.to(device), labels.to(device)
          #-- Empty the gradiant
          optimizer.zero_grad() 
          #-- Feed-forward pass
          outputs = model(images) 
          # outputs = outputs.logits
          #-- Get the predictions 
          _, preds = torch.max(outputs, 1)  
          #-- training loss calculation
          loss = loss_criterion(outputs, labels)  
          #-- Apply backpropagation to calculate gradients
          loss.backward()
          #--  Adjusting learning weights
          optimizer.step() 
          
          #-- Gather data and report
          # https://discuss.pytorch.org/t/why-do-we-multiply-loss-function-with-a-constant/112893/2
          train_loss += loss.item() * images.size(0) 
          total += labels.size(0)
          train_correct += (preds == labels).sum().item()  

          # Print results every 100 batch
          if (i + 1) % 100 == 0:
            print_ = 'Epoch {} of {}, Step: {} of {}, Training loss: {:.7f}, Training accuracy: {:.7f}, Time: {}'
            print(print_.format(epoch + 1, epochs, i + 1, len(train_loader), (train_loss / total), (train_correct / total), ts(datetime) ))
            
            # writer.add_scalar('train_loss', train_loss / (i + 1),
            #                   epoch * outputs.shape[0] + i + 1)
            # writer.add_scalar('train_accuracy', train_correct / (i + 1),
            #                   epoch * outputs.shape[0] + i + 1)
          epoch_loss = train_loss / total
          epoch_acc = train_correct / total

      #-- Print end of epoch training's results and save them in tensorboard
      print_ = 'Epoch {} of {}, Average training loss: {:.7f}, Average training accuracy: {:.7f}, Time: {}'
      print(print_.format(epoch + 1, epochs, epoch_loss,epoch_acc, ts(datetime)))
      
      # tb_writer.add_scalar('epoch_train_loss+'+parameters, epoch_loss, epoch + 1)
      # tb_writer.add_scalar('epoch_train_accuracy_'+parameters, epoch_acc, epoch + 1)
      tb_writer.add_scalars(model_name+'_epoch_loss_accuracy_'+parameters, {'train_loss':epoch_loss, 'train_accuracy':epoch_acc}, epoch + 1)
      
      print('###################### Validating {} {} model ######################'.format(model_name, parameters))
      # ts = lambda x: x.now().strftime('%d_%m_%Y__%H:%M:%S')

      # loss_criterion = nn.CrossEntropyLoss()  
      # for epoch in range(epochs):
      ######################    
      # validate the model #
      ######################
      #-- Initiate validation loss, correct validation , epoch loss, epoch accuracy, and epoch_accuracy
      val_loss = 0.0
      val_correct = 0
      total = 0
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0
      # start_time = time.time()


      with torch.no_grad(): 
        model.eval()
        for i, (val_images, val_labels) in tqdm_notebook(enumerate(val_loader)):
            #-- Pass images and labels to cuda
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            # with torch.no_grad():
                #-- Feed-forward pass
            outputs = model(val_images)  
            #-- Get the predictions
            _, predicted = torch.max(outputs.data, 1)
            #-- Validation loss calculation
            loss = loss_criterion(outputs, val_labels)
            #-- Empty the gradiant
            # optimizer.zero_grad() 

            #-- Print results every 100 batch
            val_loss += loss.item() * val_images.size(0)  
            total += val_labels.size(0)
            val_correct += (predicted == val_labels).sum().item()
            
            if (i + 1) % 100 == 0:
              #-- if there a new loss that is ower than best_val_loss , save model
              if val_epoch_loss < best_val_loss :
                print_ = 'Step: {} of {}, Validation loss: {:.7f}, Validation accuracy: {:.7f}, Time: {}'
                print(print_.format(i + 1, len(val_loader), (val_loss / total), (val_correct / total) ,ts(datetime)), end=' | ')

                print('Loss decreased from {:.7f} to {:.7f} .... Saving the model'.format(best_val_loss, val_epoch_loss))
                best_val_loss = val_epoch_loss

                #-- Saving the model state dict
                model_name_extra = '__lr {}__momentum {}__epochs {}'.format(lr, momentum, epochs)
                model_save_path = os.path.join(model_dir, model_name+model_name_extra)
                torch.save(model.state_dict(), model_save_path)

              else:
                print_ = 'Step: {} of {}, Validation loss: {:.7f}, Validation accuracy: {:.7f}, Time: {}'
                print(print_.format(i + 1, len(val_loader), (val_loss / total), (val_correct / total),ts(datetime)))

            val_epoch_loss = val_loss / total
            val_epoch_acc = val_correct / total

        #-- Print end of epoch training's results and save them in tensorboard
        print_ = 'Average validation loss: {:.7f}, Average validation accuracy: {:.7f}, Time: {}'
        print(print_.format(val_epoch_loss, val_epoch_acc,ts(datetime)))

        # tb_writer.add_scalar('epoch_val_loss_'+parameters, val_epoch_loss, epoch + 1)
        # tb_writer.add_scalar('epoch_val_accuracy_'+parameters, val_epoch_acc, epoch + 1)
        tb_writer.add_scalars(model_name+'_epoch_loss_accuracy_'+parameters, {'val_loss':val_epoch_loss, 'val_accuracy':val_epoch_acc}, epoch + 1)

  return model, model_save_path


time: 254 ms (started: 2022-05-07 00:22:04 +00:00)


# Test

In [ ]:
#-- Freeze the randomness
seed()

def model_test(model, model_name, tb_writer, parameters):  
  ts = lambda x: x.now().strftime('%d_%m_%Y__%H:%M:%S')

  print('###################### Testing {} {} model ######################'.format(model_name, parameters))
  correct = 0
  total = 0
  test_pred_labels = []
  test_actual_labels = []
  incorrect_pred = 0
  correct_pred = 0

  with torch.no_grad():
    for i, (test_images, test_labels) in tqdm_notebook(enumerate(test_loader)):
      #-- Pass images and labels to Cuda
      test_images, test_labels = test_images.to(device), test_labels.to(device)
      #-- Feed-forward pass
      outputs = model(test_images)  
      #-- Get the predictions
      _, predicted = torch.max(outputs.data, 1)
      #-- Change CPU to numpy
      temp = predicted.detach().cpu().numpy()  
      #-- Add the prediction to the test_pred_labels list
      test_pred_labels.append(temp)  
      #-- Add the acutal labels to the test_actual_labels list
      test_actual_labels.append(test_labels.detach().cpu().numpy())
      total += test_labels.size(0)
      correct += (predicted == test_labels).sum().item()
      #-- Check if the ground truth labels match the predicted labels
      if ~test_labels.all().eq(predicted.all()): #-- If so, save 10 sample images
        if incorrect_pred < 10:
          plot_results(test_images, test_labels, predicted, i, model_name, parameters, 'incorrect')
          incorrect_pred +=1
      else:
        if correct_pred < 10: #-- If not, save 10 sample images
          plot_results(test_images, test_labels, predicted, i, model_name, parameters, 'correct')
          correct_pred +=1
      #-- Print results every 100 batch
      if (i + 1) % 100 == 0:
        print('Step: {} of {}, Test accuracy: {:.7f}, Time: {}'.format(i+1, len(test_loader), (correct / total), ts(datetime) ))
  
  #-- Calculate the final accuracy and then print it
  acc = correct / total
  print('Average testing accuracy: {:.7f}, Time: {}'.format(acc, ts(datetime)))
  tb_writer.add_text(model_name+'_test_accuracy_'+parameters, str(acc))

time: 45.1 ms (started: 2022-05-07 00:22:05 +00:00)


# Plot results

In [ ]:
#-- Create a graph to compare the the actual labels with the predicted results
def plot_results(test_images, test_labels, predicted, index, model_name, parameters, correct_incorrect):
  fig = plt.figure()
  fig.suptitle('Testing {} model ({}) {} \n {}'.format(model_name, index+1, parameters, time.ctime()), fontsize=13)
  for i, (image, label, pred) in enumerate(zip(test_images, test_labels, predicted)):
    plt.subplot(1,test_images.shape[0], i+1) 
    plt.tight_layout(pad=0)
    plt.imshow(image[0].cpu(), cmap='gray')
    plt.title("Actual: {} \n Prediction: {}".format(label, pred), fontsize=10)
    plt.xticks([])
    plt.yticks([])

  #-- Save images to disk and tensorboard
  tb_writer.add_figure('Testing {} model ({}) {}'.format(model_name, index+1, parameters), fig, 0)
  image_save_path = os.path.join(output_dir, 'Test {} model_{}_{}_{}.png'.format(model_name, index+1, parameters, correct_incorrect))
  fig.savefig(image_save_path, bbox_inches='tight', pad_inches=0, transparent=True)

time: 15.2 ms (started: 2022-05-07 00:22:05 +00:00)


# Loading and Spliting MNIST Data

In [ ]:
#-- Freeze the randomness
seed()

#-- Prepare GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#-- Create data, train, and test directories
data_dir  = './datasets/MNIST'
train_dir = os.path.join(data_dir, 'train')
test_dir  = os.path.join(data_dir, 'test')

#-- Create directories for models and outputs
model_dir  = './models/MNIST'
output_dir = '/content/drive/MyDrive/DL and CV/outputs/MNIST'
Path(output_dir).mkdir(parents=True, exist_ok=True)

trans = transforms.Compose([transforms.Resize(224), #-- VGG model requires this image shape
                              transforms.RandomRotation(15), #-- images augmentation to challenge the model
                              transforms.Grayscale(num_output_channels=3), #-- VGG model requires 3 channels
                              transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) #-- Values normalization

#-- Download MNIST training data, and splitting it to training and validation
train_data = datasets.MNIST(root=train_dir, train=True, download=True, transform=trans)

#-- Split the training data to training and validation datasets.
train_indices, val_indices = train_test_split(list(range(len(train_data.targets))), test_size=5000, stratify=train_data.targets, random_state=0)
train = torch.utils.data.Subset(train_data, train_indices)
val = torch.utils.data.Subset(train_data, val_indices)

#-- Download MNIST test data
test = datasets.MNIST(root=test_dir, train=False, download=True, transform=trans)

#-- Setting training batch size:
batch_size = 4

#-- Build the training, validation, and test data loaders
train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=False)  
val_loader   = DataLoader(dataset=val,   batch_size=batch_size, shuffle=False)  
test_loader  = DataLoader(dataset=test,  batch_size=batch_size, shuffle=False) 

#-- Print out the number of salmples for training, vlidation, and test sets.
print("Number of Samples in Training Dataset: ",   len(train))
print("Number of Samples in Validation Dataset: ", len(val))
print("Number of Samples in Testing Dataset: ",    len(test))

Number of Samples in Training Dataset:  55000
Number of Samples in Validation Dataset:  5000
Number of Samples in Testing Dataset:  10000
time: 7.85 s (started: 2022-05-05 12:23:13 +00:00)


# MNIST Main

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()
    model = vgg19_bn(pretrained=False)
    model_name = 'vgg19_batch_norm'

    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/MNIST/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

###################### Training vgg19_batch_norm SGD, lr_0.0001, momentum_0 model for 5 epochs ######################


0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 13750, Training loss: 2.4102863, Training accuracy: 0.1100000, Time: 05_05_2022__12:23:59
Epoch 1 of 5, Step: 200 of 13750, Training loss: 2.3937677, Training accuracy: 0.1200000, Time: 05_05_2022__12:24:17
Epoch 1 of 5, Step: 300 of 13750, Training loss: 2.3815683, Training accuracy: 0.1250000, Time: 05_05_2022__12:24:35
Epoch 1 of 5, Step: 400 of 13750, Training loss: 2.3545620, Training accuracy: 0.1318750, Time: 05_05_2022__12:24:54
Epoch 1 of 5, Step: 500 of 13750, Training loss: 2.3390508, Training accuracy: 0.1470000, Time: 05_05_2022__12:25:12
Epoch 1 of 5, Step: 600 of 13750, Training loss: 2.3194239, Training accuracy: 0.1595833, Time: 05_05_2022__12:25:31
Epoch 1 of 5, Step: 700 of 13750, Training loss: 2.2935714, Training accuracy: 0.1717857, Time: 05_05_2022__12:25:50
Epoch 1 of 5, Step: 800 of 13750, Training loss: 2.2748892, Training accuracy: 0.1787500, Time: 05_05_2022__12:26:09
Epoch 1 of 5, Step: 900 of 13750, Training loss: 2.2563591, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.1741140, Validation accuracy: 0.9525000, Time: 05_05_2022__13:08:49 | Loss decreased from inf to 0.1755758 .... Saving the model
Step: 200 of 1250, Validation loss: 0.1690242, Validation accuracy: 0.9525000, Time: 05_05_2022__13:08:57 | Loss decreased from 0.1755758 to 0.1696449 .... Saving the model
Step: 300 of 1250, Validation loss: 0.1835095, Validation accuracy: 0.9491667, Time: 05_05_2022__13:09:04
Step: 400 of 1250, Validation loss: 0.1778716, Validation accuracy: 0.9506250, Time: 05_05_2022__13:09:09
Step: 500 of 1250, Validation loss: 0.1841958, Validation accuracy: 0.9500000, Time: 05_05_2022__13:09:14
Step: 600 of 1250, Validation loss: 0.1779419, Validation accuracy: 0.9529167, Time: 05_05_2022__13:09:19
Step: 700 of 1250, Validation loss: 0.1696081, Validation accuracy: 0.9557143, Time: 05_05_2022__13:09:25 | Loss decreased from 0.1696449 to 0.1686986 .... Saving the model
Step: 800 of 1250, Validation loss: 0.1745188, Validation accur

...

Epoch 5 of 5, Step: 100 of 13750, Training loss: 0.0250769, Training accuracy: 0.9925000, Time: 06_05_2022__03:52:18
Epoch 5 of 5, Step: 200 of 13750, Training loss: 0.0242340, Training accuracy: 0.9937500, Time: 06_05_2022__03:52:39
Epoch 5 of 5, Step: 300 of 13750, Training loss: 0.0289075, Training accuracy: 0.9925000, Time: 06_05_2022__03:52:59
Epoch 5 of 5, Step: 400 of 13750, Training loss: 0.0299597, Training accuracy: 0.9925000, Time: 06_05_2022__03:53:20
Epoch 5 of 5, Step: 500 of 13750, Training loss: 0.0349087, Training accuracy: 0.9910000, Time: 06_05_2022__03:53:41
Epoch 5 of 5, Step: 600 of 13750, Training loss: 0.0362736, Training accuracy: 0.9900000, Time: 06_05_2022__03:54:02
Epoch 5 of 5, Step: 700 of 13750, Training loss: 0.0381775, Training accuracy: 0.9892857, Time: 06_05_2022__03:54:23
Epoch 5 of 5, Step: 800 of 13750, Training loss: 0.0373144, Training accuracy: 0.9890625, Time: 06_05_2022__03:54:44
Epoch 5 of 5, Step: 900 of 13750, Training loss: 0.0388775, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.0207729, Validation accuracy: 0.9975000, Time: 06_05_2022__04:40:07
Step: 200 of 1250, Validation loss: 0.0265786, Validation accuracy: 0.9962500, Time: 06_05_2022__04:40:12
Step: 300 of 1250, Validation loss: 0.0226001, Validation accuracy: 0.9958333, Time: 06_05_2022__04:40:17
Step: 400 of 1250, Validation loss: 0.0206480, Validation accuracy: 0.9962500, Time: 06_05_2022__04:40:23
Step: 500 of 1250, Validation loss: 0.0197850, Validation accuracy: 0.9955000, Time: 06_05_2022__04:40:28
Step: 600 of 1250, Validation loss: 0.0226442, Validation accuracy: 0.9945833, Time: 06_05_2022__04:40:33
Step: 700 of 1250, Validation loss: 0.0208190, Validation accuracy: 0.9946429, Time: 06_05_2022__04:40:38
Step: 800 of 1250, Validation loss: 0.0200930, Validation accuracy: 0.9943750, Time: 06_05_2022__04:40:43
Step: 900 of 1250, Validation loss: 0.0189474, Validation accuracy: 0.9947222, Time: 06_05_2022__04:40:49
Step: 1000 of 1250, Validation loss: 0.0203403

0it [00:00, ?it/s]

Step: 100 of 2500, Test accuracy: 0.9900000, Time: 06_05_2022__04:41:16
Step: 200 of 2500, Test accuracy: 0.9887500, Time: 06_05_2022__04:41:22
Step: 300 of 2500, Test accuracy: 0.9875000, Time: 06_05_2022__04:41:27
Step: 400 of 2500, Test accuracy: 0.9862500, Time: 06_05_2022__04:41:33
Step: 500 of 2500, Test accuracy: 0.9870000, Time: 06_05_2022__04:41:38
Step: 600 of 2500, Test accuracy: 0.9858333, Time: 06_05_2022__04:41:43
Step: 700 of 2500, Test accuracy: 0.9864286, Time: 06_05_2022__04:41:48
Step: 800 of 2500, Test accuracy: 0.9859375, Time: 06_05_2022__04:41:54
Step: 900 of 2500, Test accuracy: 0.9861111, Time: 06_05_2022__04:41:59
Step: 1000 of 2500, Test accuracy: 0.9867500, Time: 06_05_2022__04:42:05
Step: 1100 of 2500, Test accuracy: 0.9877273, Time: 06_05_2022__04:42:10
Step: 1200 of 2500, Test accuracy: 0.9872917, Time: 06_05_2022__04:42:15
Step: 1300 of 2500, Test accuracy: 0.9875000, Time: 06_05_2022__04:42:21
Step: 1400 of 2500, Test accuracy: 0.9882143, Time: 06_05_20

# Loading and Spliting CIFAR Data

In [ ]:
#-- Freeze the randomness
seed()

#-- Prepare GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#-- Create data, train, and test directories
data_dir  = './datasets/CIFAR'
train_dir = os.path.join(data_dir, 'train')
test_dir  = os.path.join(data_dir, 'test')

#-- Create directories for models and outputs
model_dir  = './models/CIFAR'
output_dir = '/content/drive/MyDrive/DL and CV/outputs/CIFAR'
Path(output_dir).mkdir(parents=True, exist_ok=True)

trans = transforms.Compose([transforms.Resize(224), #-- VGG model requires this image shape
                              transforms.RandomRotation(15), #-- images augmentation to challenge the model
                              transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) #-- Values normalization

#-- Download CIFAR10 training data, and splitting it to training and validation
train_data = datasets.CIFAR10(root=train_dir, train=True, download=True, transform=trans)

#-- Split the training data to training and validation datasets.
train_indices, val_indices = train_test_split(list(range(len(train_data.targets))), test_size=5000, stratify=train_data.targets, random_state=0)
train = torch.utils.data.Subset(train_data, train_indices)
val = torch.utils.data.Subset(train_data, val_indices)

#-- Download CIFAR10 test data
test = datasets.CIFAR10(root=test_dir, train=False, download=True, transform=trans)

#-- Setting training batch size:
batch_size = 4

#-- Build the training, validation, and test data loaders
train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=False)  
val_loader   = DataLoader(dataset=val,   batch_size=batch_size, shuffle=False)  
test_loader  = DataLoader(dataset=test,  batch_size=batch_size, shuffle=False) 

print("Number of Samples in Training Dataset: ",   len(train))
print("Number of Samples in Validation Dataset: ", len(val))
print("Number of Samples in Testing Dataset: ",    len(test))

Files already downloaded and verified
Files already downloaded and verified
Number of Samples in Training Dataset:  45000
Number of Samples in Validation Dataset:  5000
Number of Samples in Testing Dataset:  10000
time: 10.5 s (started: 2022-05-07 00:22:29 +00:00)


# CIFAR Main

## Part 1

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()
    model = vgg19_bn(pretrained=False)
    model_name = 'vgg19_batch_norm'

    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/CIFAR/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

###################### Training vgg19_batch_norm SGD, lr_0.1, momentum_0 model for 5 epochs ######################


0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 11250, Training loss: 6.4188341, Training accuracy: 0.1075000, Time: 06_05_2022__04:44:02
Epoch 1 of 5, Step: 200 of 11250, Training loss: 4.4110913, Training accuracy: 0.1137500, Time: 06_05_2022__04:44:22
Epoch 1 of 5, Step: 300 of 11250, Training loss: 3.7270829, Training accuracy: 0.1091667, Time: 06_05_2022__04:44:41
Epoch 1 of 5, Step: 400 of 11250, Training loss: 3.3741683, Training accuracy: 0.1093750, Time: 06_05_2022__04:45:01
Epoch 1 of 5, Step: 500 of 11250, Training loss: 3.1603608, Training accuracy: 0.1140000, Time: 06_05_2022__04:45:21
Epoch 1 of 5, Step: 600 of 11250, Training loss: 3.0190351, Training accuracy: 0.1166667, Time: 06_05_2022__04:45:40
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.9169597, Training accuracy: 0.1214286, Time: 06_05_2022__04:46:00
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.8394547, Training accuracy: 0.1218750, Time: 06_05_2022__04:46:20
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.7802101, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 1.8017911, Validation accuracy: 0.3675000, Time: 06_05_2022__05:20:39 | Loss decreased from inf to 1.7981505 .... Saving the model
Step: 200 of 1250, Validation loss: 1.8153935, Validation accuracy: 0.3650000, Time: 06_05_2022__05:20:46
Step: 300 of 1250, Validation loss: 1.8108777, Validation accuracy: 0.3591667, Time: 06_05_2022__05:20:51
Step: 400 of 1250, Validation loss: 1.8137978, Validation accuracy: 0.3587500, Time: 06_05_2022__05:20:57
Step: 500 of 1250, Validation loss: 1.8246531, Validation accuracy: 0.3530000, Time: 06_05_2022__05:21:02
Step: 600 of 1250, Validation loss: 1.8255114, Validation accuracy: 0.3500000, Time: 06_05_2022__05:21:07
Step: 700 of 1250, Validation loss: 1.8291330, Validation accuracy: 0.3482143, Time: 06_05_2022__05:21:12
Step: 800 of 1250, Validation loss: 1.8273261, Validation accuracy: 0.3443750, Time: 06_05_2022__05:21:17
Step: 900 of 1250, Validation loss: 1.8280932, Validation accuracy: 0.3430556, Time: 06_05_

...

Epoch 1 of 5, Step: 100 of 11250, Training loss: 8.1320510, Training accuracy: 0.1050000, Time: 06_05_2022__11:20:36
Epoch 1 of 5, Step: 200 of 11250, Training loss: 5.2119090, Training accuracy: 0.1025000, Time: 06_05_2022__11:20:57
Epoch 1 of 5, Step: 300 of 11250, Training loss: 4.2446527, Training accuracy: 0.1058333, Time: 06_05_2022__11:21:18
Epoch 1 of 5, Step: 400 of 11250, Training loss: 3.7593509, Training accuracy: 0.1075000, Time: 06_05_2022__11:21:39
Epoch 1 of 5, Step: 500 of 11250, Training loss: 3.4682180, Training accuracy: 0.1165000, Time: 06_05_2022__11:21:59
Epoch 1 of 5, Step: 600 of 11250, Training loss: 3.2689229, Training accuracy: 0.1175000, Time: 06_05_2022__11:22:20
Epoch 1 of 5, Step: 700 of 11250, Training loss: 3.1272837, Training accuracy: 0.1200000, Time: 06_05_2022__11:22:41
Epoch 1 of 5, Step: 800 of 11250, Training loss: 3.0211651, Training accuracy: 0.1228125, Time: 06_05_2022__11:23:02
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.9354883, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 1.7309989, Validation accuracy: 0.4075000, Time: 06_05_2022__11:59:22 | Loss decreased from inf to 1.7293160 .... Saving the model
Step: 200 of 1250, Validation loss: 1.7528904, Validation accuracy: 0.3900000, Time: 06_05_2022__11:59:29
Step: 300 of 1250, Validation loss: 1.7488454, Validation accuracy: 0.3916667, Time: 06_05_2022__11:59:35
Step: 400 of 1250, Validation loss: 1.7391092, Validation accuracy: 0.3943750, Time: 06_05_2022__11:59:40
Step: 500 of 1250, Validation loss: 1.7432625, Validation accuracy: 0.3940000, Time: 06_05_2022__11:59:45
Step: 600 of 1250, Validation loss: 1.7330499, Validation accuracy: 0.4020833, Time: 06_05_2022__11:59:50
Step: 700 of 1250, Validation loss: 1.7315883, Validation accuracy: 0.4028571, Time: 06_05_2022__11:59:56
Step: 800 of 1250, Validation loss: 1.7266894, Validation accuracy: 0.4031250, Time: 06_05_2022__12:00:01 | Loss decreased from 1.7293160 to 1.7271185 .... Saving the model
Step: 900 of 1250, Valid

0it [00:00, ?it/s]

Epoch 2 of 5, Step: 100 of 11250, Training loss: 1.7689699, Training accuracy: 0.3575000, Time: 06_05_2022__12:00:50
Epoch 2 of 5, Step: 200 of 11250, Training loss: 1.7962476, Training accuracy: 0.3562500, Time: 06_05_2022__12:01:11
Epoch 2 of 5, Step: 300 of 11250, Training loss: 1.8190033, Training accuracy: 0.3466667, Time: 06_05_2022__12:01:31
Epoch 2 of 5, Step: 400 of 11250, Training loss: 1.8172240, Training accuracy: 0.3481250, Time: 06_05_2022__12:01:52
Epoch 2 of 5, Step: 500 of 11250, Training loss: 1.8133751, Training accuracy: 0.3490000, Time: 06_05_2022__12:02:13
Epoch 2 of 5, Step: 600 of 11250, Training loss: 1.7989827, Training accuracy: 0.3516667, Time: 06_05_2022__12:02:34
Epoch 2 of 5, Step: 700 of 11250, Training loss: 1.8051714, Training accuracy: 0.3503571, Time: 06_05_2022__12:02:55
Epoch 2 of 5, Step: 800 of 11250, Training loss: 1.7947222, Training accuracy: 0.3509375, Time: 06_05_2022__12:03:16
Epoch 2 of 5, Step: 900 of 11250, Training loss: 1.7948665, Trai

## Part 2


In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()
    model = vgg19_bn(pretrained=False)
    model_name = 'vgg19_batch_norm'

    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/CIFAR/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

###################### Training vgg19_batch_norm SGD, lr_0.1, momentum_0.6 model for 5 epochs ######################


0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 11250, Training loss: 8.1320510, Training accuracy: 0.1050000, Time: 07_05_2022__00:24:09
Epoch 1 of 5, Step: 200 of 11250, Training loss: 5.2119090, Training accuracy: 0.1025000, Time: 07_05_2022__00:24:29
Epoch 1 of 5, Step: 300 of 11250, Training loss: 4.2446527, Training accuracy: 0.1058333, Time: 07_05_2022__00:24:50
Epoch 1 of 5, Step: 400 of 11250, Training loss: 3.7593509, Training accuracy: 0.1075000, Time: 07_05_2022__00:25:10
Epoch 1 of 5, Step: 500 of 11250, Training loss: 3.4682180, Training accuracy: 0.1165000, Time: 07_05_2022__00:25:31
Epoch 1 of 5, Step: 600 of 11250, Training loss: 3.2689229, Training accuracy: 0.1175000, Time: 07_05_2022__00:25:51
Epoch 1 of 5, Step: 700 of 11250, Training loss: 3.1272837, Training accuracy: 0.1200000, Time: 07_05_2022__00:26:11
Epoch 1 of 5, Step: 800 of 11250, Training loss: 3.0211651, Training accuracy: 0.1228125, Time: 07_05_2022__00:26:32
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.9354883, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 1.7309989, Validation accuracy: 0.4075000, Time: 07_05_2022__01:02:06 | Loss decreased from inf to 1.7293160 .... Saving the model
Step: 200 of 1250, Validation loss: 1.7528904, Validation accuracy: 0.3900000, Time: 07_05_2022__01:02:13
Step: 300 of 1250, Validation loss: 1.7488454, Validation accuracy: 0.3916667, Time: 07_05_2022__01:02:18
Step: 400 of 1250, Validation loss: 1.7391092, Validation accuracy: 0.3943750, Time: 07_05_2022__01:02:24
Step: 500 of 1250, Validation loss: 1.7432625, Validation accuracy: 0.3940000, Time: 07_05_2022__01:02:29
Step: 600 of 1250, Validation loss: 1.7330499, Validation accuracy: 0.4020833, Time: 07_05_2022__01:02:34
Step: 700 of 1250, Validation loss: 1.7315883, Validation accuracy: 0.4028571, Time: 07_05_2022__01:02:39
Step: 800 of 1250, Validation loss: 1.7266894, Validation accuracy: 0.4031250, Time: 07_05_2022__01:02:44 | Loss decreased from 1.7293160 to 1.7271185 .... Saving the model
Step: 900 of 1250, Valid

...

Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.4943796, Training accuracy: 0.1000000, Time: 07_05_2022__20:07:27
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.4559565, Training accuracy: 0.1125000, Time: 07_05_2022__20:07:47
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.4331661, Training accuracy: 0.1208333, Time: 07_05_2022__20:08:07
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.4002925, Training accuracy: 0.1437500, Time: 07_05_2022__20:08:26
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.3725303, Training accuracy: 0.1505000, Time: 07_05_2022__20:08:46
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.3560750, Training accuracy: 0.1533333, Time: 07_05_2022__20:09:06
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.3351648, Training accuracy: 0.1603571, Time: 07_05_2022__20:09:25
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.3172936, Training accuracy: 0.1618750, Time: 07_05_2022__20:09:45
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.3069272, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 1.4568086, Validation accuracy: 0.4775000, Time: 07_05_2022__20:43:58 | Loss decreased from inf to 1.4565268 .... Saving the model
Step: 200 of 1250, Validation loss: 1.4466892, Validation accuracy: 0.4762500, Time: 07_05_2022__20:44:05 | Loss decreased from 1.4565268 to 1.4494065 .... Saving the model
Step: 300 of 1250, Validation loss: 1.4721930, Validation accuracy: 0.4575000, Time: 07_05_2022__20:44:12
Step: 400 of 1250, Validation loss: 1.4652821, Validation accuracy: 0.4631250, Time: 07_05_2022__20:44:18
Step: 500 of 1250, Validation loss: 1.4661676, Validation accuracy: 0.4680000, Time: 07_05_2022__20:44:23
Step: 600 of 1250, Validation loss: 1.4586979, Validation accuracy: 0.4700000, Time: 07_05_2022__20:44:28
Step: 700 of 1250, Validation loss: 1.4509881, Validation accuracy: 0.4800000, Time: 07_05_2022__20:44:33
Step: 800 of 1250, Validation loss: 1.4404687, Validation accuracy: 0.4856250, Time: 07_05_2022__20:44:39 | Loss decreased from 1.

0it [00:00, ?it/s]

Epoch 2 of 5, Step: 100 of 11250, Training loss: 1.5088709, Training accuracy: 0.4600000, Time: 07_05_2022__20:45:24
Epoch 2 of 5, Step: 200 of 11250, Training loss: 1.5490343, Training accuracy: 0.4437500, Time: 07_05_2022__20:45:44
Epoch 2 of 5, Step: 300 of 11250, Training loss: 1.5568463, Training accuracy: 0.4391667, Time: 07_05_2022__20:46:03
Epoch 2 of 5, Step: 400 of 11250, Training loss: 1.5469770, Training accuracy: 0.4443750, Time: 07_05_2022__20:46:23
Epoch 2 of 5, Step: 500 of 11250, Training loss: 1.5498705, Training accuracy: 0.4415000, Time: 07_05_2022__20:46:43


## Part 3

In [ ]:
#-- Parameters
epochs = 5

for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
  for mom in [0, 0.3, 0.6, 0.9]:
    #-- Freeze the randomness
    seed()
    model = vgg19_bn(pretrained=False)
    model_name = 'vgg19_batch_norm'

    lr = learning_rate
    momentum = mom
    parameters = 'SGD, lr_{}, momentum_{}'.format(lr, momentum)

    #-- Initiating a tensorboard writer that contains all logs for training, validation, and testing
    tb_writer = SummaryWriter('./models/CIFAR/runs', filename_suffix='_'+model_name+'_'+parameters)

    #-- Create the model's critrion loss function and a optimization function 
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    #-- Train and valiate the model
    model, model_save_path = model_train(model, model_name, lr, momentum, optimizer, epochs, tb_writer, parameters)

    #-- Load the best saved model for testing
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)

    #-- Set the model mode to evaluation to prepare for testing
    model.eval()
    model_test(model, model_name, tb_writer, parameters) 

    #-- Close the Tensoboard writer
    tb_writer.close()

    #-- Delete the model
    del model

###################### Training vgg19_batch_norm SGD, lr_0.001, momentum_0.3 model for 5 epochs ######################


0it [00:00, ?it/s]

Epoch 1 of 5, Step: 100 of 11250, Training loss: 2.4975640, Training accuracy: 0.0950000, Time: 08_05_2022__12:22:01
Epoch 1 of 5, Step: 200 of 11250, Training loss: 2.4752028, Training accuracy: 0.1112500, Time: 08_05_2022__12:22:12
Epoch 1 of 5, Step: 300 of 11250, Training loss: 2.4464085, Training accuracy: 0.1158333, Time: 08_05_2022__12:22:23
Epoch 1 of 5, Step: 400 of 11250, Training loss: 2.4117174, Training accuracy: 0.1262500, Time: 08_05_2022__12:22:34
Epoch 1 of 5, Step: 500 of 11250, Training loss: 2.3815575, Training accuracy: 0.1345000, Time: 08_05_2022__12:22:45
Epoch 1 of 5, Step: 600 of 11250, Training loss: 2.3607387, Training accuracy: 0.1429167, Time: 08_05_2022__12:22:56
Epoch 1 of 5, Step: 700 of 11250, Training loss: 2.3381120, Training accuracy: 0.1503571, Time: 08_05_2022__12:23:06
Epoch 1 of 5, Step: 800 of 11250, Training loss: 2.3141232, Training accuracy: 0.1571875, Time: 08_05_2022__12:23:17
Epoch 1 of 5, Step: 900 of 11250, Training loss: 2.3007561, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 1.4237943, Validation accuracy: 0.4850000, Time: 08_05_2022__12:42:19 | Loss decreased from inf to 1.4247893 .... Saving the model
Step: 200 of 1250, Validation loss: 1.4252489, Validation accuracy: 0.4912500, Time: 08_05_2022__12:42:25
Step: 300 of 1250, Validation loss: 1.4436732, Validation accuracy: 0.4750000, Time: 08_05_2022__12:42:29
Step: 400 of 1250, Validation loss: 1.4403692, Validation accuracy: 0.4768750, Time: 08_05_2022__12:42:32
Step: 500 of 1250, Validation loss: 1.4386434, Validation accuracy: 0.4825000, Time: 08_05_2022__12:42:36
Step: 600 of 1250, Validation loss: 1.4307907, Validation accuracy: 0.4825000, Time: 08_05_2022__12:42:40
Step: 700 of 1250, Validation loss: 1.4195108, Validation accuracy: 0.4871429, Time: 08_05_2022__12:42:43 | Loss decreased from 1.4247893 to 1.4197062 .... Saving the model
Step: 800 of 1250, Validation loss: 1.4101916, Validation accuracy: 0.4896875, Time: 08_05_2022__12:42:49 | Loss decreased from 1.

...

Epoch 5 of 5, Step: 100 of 11250, Training loss: 0.9347558, Training accuracy: 0.6775000, Time: 09_05_2022__00:42:18
Epoch 5 of 5, Step: 200 of 11250, Training loss: 0.9494213, Training accuracy: 0.6712500, Time: 09_05_2022__00:42:29
Epoch 5 of 5, Step: 300 of 11250, Training loss: 0.9655393, Training accuracy: 0.6550000, Time: 09_05_2022__00:42:40
Epoch 5 of 5, Step: 400 of 11250, Training loss: 0.9426223, Training accuracy: 0.6625000, Time: 09_05_2022__00:42:52
Epoch 5 of 5, Step: 500 of 11250, Training loss: 0.9472997, Training accuracy: 0.6605000, Time: 09_05_2022__00:43:03
Epoch 5 of 5, Step: 600 of 11250, Training loss: 0.9482928, Training accuracy: 0.6612500, Time: 09_05_2022__00:43:14
Epoch 5 of 5, Step: 700 of 11250, Training loss: 0.9597937, Training accuracy: 0.6592857, Time: 09_05_2022__00:43:25
Epoch 5 of 5, Step: 800 of 11250, Training loss: 0.9532236, Training accuracy: 0.6637500, Time: 09_05_2022__00:43:36
Epoch 5 of 5, Step: 900 of 11250, Training loss: 0.9607317, Trai

0it [00:00, ?it/s]

Step: 100 of 1250, Validation loss: 0.8817830, Validation accuracy: 0.6775000, Time: 09_05_2022__01:03:01 | Loss decreased from 0.9465317 to 0.8870669 .... Saving the model
Step: 200 of 1250, Validation loss: 0.8675649, Validation accuracy: 0.6725000, Time: 09_05_2022__01:03:07 | Loss decreased from 0.8870669 to 0.8699546 .... Saving the model
Step: 300 of 1250, Validation loss: 0.8963527, Validation accuracy: 0.6733333, Time: 09_05_2022__01:03:13
Step: 400 of 1250, Validation loss: 0.8796912, Validation accuracy: 0.6843750, Time: 09_05_2022__01:03:17
Step: 500 of 1250, Validation loss: 0.8982340, Validation accuracy: 0.6805000, Time: 09_05_2022__01:03:21
Step: 600 of 1250, Validation loss: 0.8868633, Validation accuracy: 0.6858333, Time: 09_05_2022__01:03:25
Step: 700 of 1250, Validation loss: 0.8765842, Validation accuracy: 0.6921429, Time: 09_05_2022__01:03:28
Step: 800 of 1250, Validation loss: 0.8613843, Validation accuracy: 0.6950000, Time: 09_05_2022__01:03:32 | Loss decreased f

0it [00:00, ?it/s]

Step: 100 of 2500, Test accuracy: 0.7000000, Time: 09_05_2022__01:04:02
Step: 200 of 2500, Test accuracy: 0.7087500, Time: 09_05_2022__01:04:06
Step: 300 of 2500, Test accuracy: 0.7075000, Time: 09_05_2022__01:04:10
Step: 400 of 2500, Test accuracy: 0.6937500, Time: 09_05_2022__01:04:13
Step: 500 of 2500, Test accuracy: 0.6875000, Time: 09_05_2022__01:04:17
Step: 600 of 2500, Test accuracy: 0.6841667, Time: 09_05_2022__01:04:21
Step: 700 of 2500, Test accuracy: 0.6814286, Time: 09_05_2022__01:04:24
Step: 800 of 2500, Test accuracy: 0.6834375, Time: 09_05_2022__01:04:28
Step: 900 of 2500, Test accuracy: 0.6833333, Time: 09_05_2022__01:04:32
Step: 1000 of 2500, Test accuracy: 0.6825000, Time: 09_05_2022__01:04:35
Step: 1100 of 2500, Test accuracy: 0.6856818, Time: 09_05_2022__01:04:39
Step: 1200 of 2500, Test accuracy: 0.6852083, Time: 09_05_2022__01:04:42
Step: 1300 of 2500, Test accuracy: 0.6863462, Time: 09_05_2022__01:04:46
Step: 1400 of 2500, Test accuracy: 0.6869643, Time: 09_05_20